In [3]:
import tensorflow as tf
import os
import shutil

/Users/mrbinit/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
#define the paths
base_dir = "/Users/mrbinit/Downloads/aclImdb"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
val_dir = os.path.join(base_dir, "val")

#create a validation directory
os.makedirs(val_dir, exist_ok= True)

#split the test data  into validation and test set 
for label in ['pos', 'neg']:
    label_dir = os.path.join(test_dir, label)
    val_label_dir = os.path.join(val_dir, label)
    os.makedirs(val_label_dir, exist_ok=True)

#get the list of files in the label directory
files = os.listdir(label_dir)

for file in files[:15000]:
    shutil.move(os.path.join(label_dir, file), os.path.join(val_dir, file))
